In [1]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [2]:
import numpy as np

In [3]:
!unzip /content/drive/MyDrive/HACKATHON/archive.zip

Archive:  /content/drive/MyDrive/HACKATHON/archive.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import pandas as pd

df = pd.read_csv("/content/train.csv")

In [5]:
test_df = pd.read_csv("/content/test.csv")

In [6]:
# !pip install git+https://github.com/openai/CLIP.git
# import clip

In [7]:
import io
import ast # this is

In [8]:
def convert_bytes_to_Image(byte_array) :
    return Image.open(io.BytesIO((ast.literal_eval(byte_array)['bytes'])))

In [9]:
images = []
texts = []

for i in range(len(df)):

  img = convert_bytes_to_Image(df['image'][i])

  l_captions =  df['captions'][i].replace('[','').replace(']','')
  list_captions=[element.strip() for element in l_captions.split('\n')]

  images.append(img)
  texts.append(list_captions)


data = {'images':images, "captions":texts}

image_train_df = pd.DataFrame(data)

In [10]:
images = []
texts = []

for i in range(len(test_df)):

  img = convert_bytes_to_Image(test_df['image'][i])

  l_captions =  test_df['captions'][i].replace('[','').replace(']','')
  list_captions=[element.strip() for element in l_captions.split('\n')]

  images.append(img)
  texts.append(list_captions)


data = {'images':images, "captions":texts}

image_test_df = pd.DataFrame(data)

In [11]:
image_train_df.head()

,images,captions
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,['Many aircraft are parked next to a long buil...
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"['some planes are parked in an airport.', 'the..."
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,['Many aircraft are parked in an airport near ...
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,['Many aircraft are parked near a large buildi...
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,['several buildings and green trees are around...


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")


In [13]:
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class ImageCaptioningDataset(Dataset):
    def __init__(self, data, tokenizer, feature_extractor):
        self.data = data
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        try:
            image_path, captions = self.data[idx]
        except Exception as e:
            print(f"Error loading item at index {idx}: {e}")
            return self.__getitem__((idx + 1) % len(self.data))  # Try next item

        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Error opening image at index {idx}: {e}")
            image = Image.new("RGB", (224, 224))  # Create a dummy image
            return self._process_item(image, captions)

        return self._process_item(image, captions)

    def _process_item(self, image, captions):
        try:
            inputs = self.feature_extractor(images=image, return_tensors="pt")
            input_ids = inputs["pixel_values"].squeeze(0)

            # Handle cases where there are no captions
            if len(captions) == 0:
                captions = ["[UNK]"]

            caption_ids = [self.tokenizer.encode(caption, add_special_tokens=True) for caption in captions]
            return input_ids, caption_ids

        except Exception as e:
            print(f"Error processing item: {e}")
            image = Image.new("RGB", (224, 224))  # Create a dummy image
            return self._process_item(image, captions)

# Assuming train_data and test_data are lists of tuples (image_path, captions)
train_dataset = ImageCaptioningDataset(image_train_df, tokenizer, feature_extractor)
test_dataset = ImageCaptioningDataset(image_test_df, tokenizer, feature_extractor)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [15]:
from transformers import AdamW
from tqdm import tqdm

In [59]:
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.shape[-1]), targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}")

Epoch 1/10:   0%|          | 0/4367 [00:00<?, ?it/s]

Error loading item at index 3573: 3573
Error loading item at index 3574: 3574
Error loading item at index 3575: 3575
Error loading item at index 3576: 3576
Error loading item at index 3577: 3577
Error loading item at index 3578: 3578
Error loading item at index 3579: 3579
Error loading item at index 3580: 3580
Error loading item at index 3581: 3581
Error loading item at index 3582: 3582
Error loading item at index 3583: 3583
Error loading item at index 3584: 3584
Error loading item at index 3585: 3585
Error loading item at index 3586: 3586
Error loading item at index 3587: 3587
Error loading item at index 3588: 3588
Error loading item at index 3589: 3589
Error loading item at index 3590: 3590
Error loading item at index 3591: 3591
Error loading item at index 3592: 3592
Error loading item at index 3593: 3593
Error loading item at index 3594: 3594
Error loading item at index 3595: 3595
Error loading item at index 3596: 3596
Error loading item at index 3597: 3597
Error loading item at ind

Epoch 1/10:   0%|          | 0/4367 [00:00<?, ?it/s]


Error loading item at index 4272: 4272
Error loading item at index 4273: 4273
Error loading item at index 4274: 4274
Error loading item at index 4275: 4275
Error loading item at index 4276: 4276
Error loading item at index 4277: 4277
Error loading item at index 4278: 4278
Error loading item at index 4279: 4279
Error loading item at index 4280: 4280
Error loading item at index 4281: 4281
Error loading item at index 4282: 4282
Error loading item at index 4283: 4283
Error loading item at index 4284: 4284
Error loading item at index 4285: 4285
Error loading item at index 4286: 4286
Error loading item at index 4287: 4287
Error loading item at index 4288: 4288
Error loading item at index 4289: 4289
Error loading item at index 4290: 4290
Error loading item at index 4291: 4291
Error loading item at index 4292: 4292
Error loading item at index 4293: 4293
Error loading item at index 4294: 4294
Error loading item at index 4295: 4295
Error loading item at index 4296: 4296
Error loading item at in


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 3573

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-59-24b9d2e99183>", line 18, in __getitem__
    image_path, captions = self.data[idx]
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py", line 3807, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/usr/local/lib/python3.10/dist-packages/pandas/co

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Exception ignored in: <function tqdm.__del__ at 0x79d358a41bd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1147, in __del__
    def __del__(self):
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
def predict_step(model, device, tokenizer, feature_extractor, image_paths):
    model.eval()
    model.to(device)

    predictions = []

    for image_path in image_paths:
        image = Image.open(image_path)
        if image.mode != "RGB":
            image = image.convert(mode="RGB")

        inputs = feature_extractor(images=image, return_tensors="pt")
        inputs = inputs.pixel_values.to(device)

        with torch.no_grad():
            output_ids = model.generate(inputs, **gen_kwargs)

        preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        preds = [pred.strip() for pred in preds]
        predictions.append(preds)

    return predictions

# Load the trained model
model_path = "./results"  # Update with your actual model path
model = VisionEncoderDecoderModel.from_pretrained(model_path)

# Use the modified predict_step function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predictions = predict_step(model, device, tokenizer, feature_extractor, ['sample2.jpg'])

print(predictions)